In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from desilike.theories.galaxy_clustering import FOLPSAXTracerPowerSpectrumMultipoles, DirectPowerSpectrumTemplate, FixedPowerSpectrumTemplate

template = DirectPowerSpectrumTemplate(fiducial = 'DESI', z = 0.5)
theory = FOLPSAXTracerPowerSpectrumMultipoles(template = template,tracer = 'LRG', prior_basis = 'physical')

In [3]:
template.params['n_s'].update(fixed = False)
template.params['n_s'].update(prior={'dist':'norm','loc':0.9649, 'scale':0.048}) #Planck width x10
theory.params['b3p'].update(fixed = False)

In [4]:
#Data from Y1 clustering products

import numpy as np
from pypower import PowerSpectrumMultipoles, BaseMatrix
from desilike.observables import ObservableCovariance

data_fn = '/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/baseline_2pt/pk/corrected/pkpoles_corrected_LRG_SGC_z0.4-0.6_thetacut0.05.npy'
wmatrix_fn = '/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/baseline_2pt/pk/wmatrix_smooth_LRG_SGC_z0.4-0.6_thetacut0.05.npy'
covariance_fn = '/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/cov_2pt/ezmock/v1/covariance_power_LRG_SGC_z0.4-0.6_default_FKP_lin_thetacut0.05.npy'

In [5]:
covariance = ObservableCovariance.load(covariance_fn)
covariance = covariance.select(xlim = (0.02, 0.2), projs = [0,2])

In [6]:
#Defining an observable without window function
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable

observable = TracerPowerSpectrumMultipolesObservable(data=data_fn, 
                                                     covariance=covariance,
                                                     klim={ell: [0.02, 0.2, 0.005] for ell in [0,2]},
                                                     theory=theory,
                                                     wmatrix = wmatrix_fn,
                                                     kin = np.arange(0.001, 0.35, 0.001),
                                                     )

In [7]:
observable.covariance.shape

(72, 72)

In [7]:
from desilike.emulators import EmulatedCalculator, Emulator, TaylorEmulatorEngine
theory = observable.wmatrix.theory
emulator = Emulator(theory.pt, engine=TaylorEmulatorEngine(method = 'finite', order = 1))
emulator.save('Emulator/FOLPSAX_mf_Taylor_o4_LRG1')
emulator.set_samples()
emulator.fit()

KeyboardInterrupt: 

In [ ]:
theory.init.update(pt = emulator.to_calculator())

In [8]:
theory.params['alpha0p'].update(derived = '.marg')
theory.params['alpha2p'].update(derived = '.marg')
theory.params['alpha4p'].update(derived = '.marg')
theory.params['sn0p'].update(derived = '.marg')
theory.params['sn2p'].update(derived = '.marg')

In [9]:
# Update namespace of bias parameters (to have one parameter per tracer / z-bin)
for param in theory.init.params:
    # Update latex just to have better labels
    param.update(namespace='{}'.format('LRG1_SGC'),
                 latex=param.latex(namespace=#'\mathrm{{pre}},
                                   '\mathrm{{{}}}, {:d}'.format('LRG', 0), inline=False))

In [10]:
from desilike.likelihoods import ObservablesGaussianLikelihood
from desilike import setup_logging

setup_logging()

likelihood = ObservablesGaussianLikelihood(observables = [observable])

In [11]:
template2 = DirectPowerSpectrumTemplate(fiducial = 'DESI', z = 0.5)
theory2 = FOLPSAXTracerPowerSpectrumMultipoles(template = template2, tracer = 'LRG', prior_basis = 'physical')

In [12]:
template2.params['n_s'].update(fixed = False)
template2.params['n_s'].update(prior={'dist':'norm','loc':0.9649, 'scale':0.048}) #Planck width x10
theory2.params['b3p'].update(fixed = False)

In [13]:
data_fn_2 = '/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/baseline_2pt/pk/corrected/pkpoles_corrected_LRG_NGC_z0.4-0.6_thetacut0.05.npy'


#data_fn_2 = '/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/baseline_2pt/pk/corrected/pkpoles_corrected_LRG_NGC_z0.8-1.1_thetacut0.05.npy'
wmatrix_fn_2 = '/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/baseline_2pt/pk/wmatrix_smooth_LRG_NGC_z0.4-0.6_thetacut0.05.npy'
covariance_fn_2 = '/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/cov_2pt/ezmock/v1/covariance_power_LRG_NGC_z0.4-0.6_default_FKP_lin_thetacut0.05.npy'

In [14]:
covariance_2 = ObservableCovariance.load(covariance_fn_2)
covariance_2 = covariance_2.select(xlim = (0.02, 0.2), projs = [0,2])

[000004.02] [0/1] 07-17 10:32  ObservableCovariance      INFO     Loading /global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/cov_2pt/ezmock/v1/covariance_power_LRG_NGC_z0.4-0.6_default_FKP_lin_thetacut0.05.npy.


In [15]:
observable2 = TracerPowerSpectrumMultipolesObservable(data=data_fn_2, 
                                                     covariance=covariance_2,
                                                     klim={ell: [0.02, 0.2, 0.005] for ell in [0,2]},
                                                     theory=theory2,
                                                     wmatrix = wmatrix_fn_2,
                                                     kin = np.arange(0.001, 0.35, 0.001),
                                                     )

In [34]:
observable.data[0].shape

(36,)

In [ ]:
theory2 = observable2.wmatrix.theory
theory2.init.update(pt = emulator.to_calculator())

In [16]:
theory2.params['alpha0p'].update(derived = '.marg')
theory2.params['alpha2p'].update(derived = '.marg')
theory2.params['alpha4p'].update(derived = '.marg')
theory2.params['sn0p'].update(derived = '.marg')
theory2.params['sn2p'].update(derived = '.marg')

In [17]:
# Update namespace of bias parameters (to have one parameter per tracer / z-bin)
for param in theory2.init.params:
    # Update latex just to have better labels
    param.update(namespace='{}'.format('LRG1_NGC'),
                 latex=param.latex(namespace=#'\mathrm{{pre}},
                                   '\mathrm{{{}}}, {:d}'.format('LRG', 0), inline=False))

In [18]:
likelihood2 = ObservablesGaussianLikelihood(observables = [observable2])

In [20]:
from desilike.likelihoods import SumLikelihood

Likelihood = SumLikelihood(likelihoods = (likelihood, likelihood2))

Likelihood()

[000030.48] [0/1] 07-17 10:32  TracerPowerSpectrumMultipolesObservable INFO     Loading 1 file ['/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/baseline_2pt/pk/corrected/pkpoles_corrected_LRG_NGC_z0.4-0.6_thetacut0.05.npy'].
[000031.23] [0/1] 07-17 10:32  MeshFFTWindow             INFO     Loading /global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/baseline_2pt/pk/wmatrix_smooth_LRG_NGC_z0.4-0.6_thetacut0.05.npy.
[000032.26] [0/1] 07-17 10:32  BaseMatrix                INFO     Loading /global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/baseline_2pt/pk/wmatrix_smooth_LRG_NGC_z0.4-0.6_thetacut0.05.npy.
[000042.98] [0/1] 07-17 10:32  ObservablesGaussianLikelihood INFO     Covariance matrix with 72 points built from 1000 observations.
[000042.98] [0/1] 07-17 10:32  ObservablesGaussianLikelihood INFO     ...resulting in a Hartlap 2007 factor of 0.9269.
[000052.69] [0/1] 07-17 10:32  Observable

/global/common/software/desi/users/adematti/perlmutter/cosmodesiconda/20240118-1.0.0/code/desilike/hmc/lib/python3.10/site-packages/desilike/base.py:419: UserWarning: Derived parameter loglikelihood of <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7f4810736230> is already derived in <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7f481079c640>.
  if self.mpicomm.rank == 0: warnings.warn(msg)
/global/common/software/desi/users/adematti/perlmutter/cosmodesiconda/20240118-1.0.0/code/desilike/hmc/lib/python3.10/site-packages/desilike/base.py:419: UserWarning: Derived parameter logprior of <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7f4810736230> is already derived in <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7f481079c640>.
  if self.mpicomm.rank == 0: warnings.warn(msg)
/global/common/software/desi/users/adematti/perlmutter/cosmodesiconda/20240118-1.0.0/code/desilike/hmc/lib/python3.10/site-pack

Array(-153.17139117, dtype=float64)

In [ ]:
from desilike.samplers import EmceeSampler

sampler = EmceeSampler(Likelihood,save_fn = 'Chains/LRG1_NGC_SGC_ell02_kmax0.2')
sampler.run(check={'max_eigen_gr': 0.03})

In [ ]:
from desilike.samples import Chain, plotting

chain = Chain.load('Chains/LRG_N_plus_S_script.npy').remove_burnin(0.3)

In [ ]:
a = np.array(chain)

In [ ]:
a = a[0:14, ::, ::]

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import getdist
import IPython
from getdist import plots, MCSamples

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#chain2_ = reader.get_chain(discard=discard, flat=True, thin=1)

labels = [r'$h$', r'$\omega_{cdm}$', r'$\omega_b$', r'$\log{10^{10}A_s}$', r'$n_s$', r'$b_1$', r'$b_2$', r'$b_s$', r'$b_3$', r'$\alpha_0$', r'$\alpha_2$', r'$\alpha_4$', r'$sn_0$', r'$sn_2$']

samples = MCSamples(samples=a.T, names = labels)# ranges={r'$fR0$':(1e-10, 0.1)})
#plt.rcParams['text.usetex'] = True
s3 = samples.copy(settings={'mult_bias_correction_order':1,
                       'smooth_scale_2D':0.55, 
                       'smooth_scale_1D':0.55})

g = plots.get_subplot_plotter()

g.triangle_plot([s3], 
                [r'$h$', r'$\omega_{cdm}$', r'$\omega_b$', r'$\log{10^{10}A_s}$', r'$n_s$'], 
                filled=True,
               contour_colors=['orange', 'green'],
               markers = [cosmo['h'], cosmo['omega_cdm'], cosmo['omega_b'], cosmo['logA'], cosmo['n_s']],
               legend_labels = [r'LRG_NGC+SGC'])
#plt.suptitle(r'LRG_NGC', fontsize = 20);


plt.savefig('Graphs/Fit_DESIY1_LRG_NGCplusSGC')

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import getdist
import IPython
from getdist import plots, MCSamples

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#chain2_ = reader.get_chain(discard=discard, flat=True, thin=1)

labels = [r'$h$', r'$\omega_{cdm}$', r'$\omega_b$', r'$\log{10^{10}A_s}$', r'$n_s$', r'$b_1$', r'$b_2$', r'$b_s$', r'$b_3$', r'$\alpha_0$', r'$\alpha_2$', r'$\alpha_4$', r'$sn_0$', r'$sn_2$']

samples = MCSamples(samples=a.T, names = labels)# ranges={r'$fR0$':(1e-10, 0.1)})
#plt.rcParams['text.usetex'] = True
s3 = samples.copy(settings={'mult_bias_correction_order':1,
                       'smooth_scale_2D':0.55, 
                       'smooth_scale_1D':0.55})

g = plots.get_subplot_plotter()
g.triangle_plot([s3], 
                [r'$h$', r'$\omega_{cdm}$', r'$\omega_b$', r'$\log{10^{10}A_s}$', r'$n_s$', r'$b_1$', r'$b_2$', r'$b_s$', r'$b_3$', r'$\alpha_0$', r'$\alpha_2$', r'$\alpha_4$', r'$sn_0$', r'$sn_2$'], 
                filled=True,
               contour_colors=['orange'],
               markers = [cosmo['h'], cosmo['omega_cdm'], cosmo['omega_b'], cosmo['logA'], cosmo['n_s']],
               legend_labels = ['LRG_NGC+SGC'])
#plt.suptitle(r'LRG_NGC', fontsize = 20);


plt.savefig('Graphs/Fit_DESIY1_LRG_NGCplus_SGC')

In [ ]:
labels = [r'$h$', r'$\omega_{cdm}$', r'$\omega_b$', r'$\log{A}$', r'$n_s$', r'$b_1$', r'$b_2$', r'$b_s$', r'$b_3$', r'$\alpha_0$', r'$\alpha_2$', r'$\alpha_4$', r'$sn_0$', r'$sn_2$']
fig, axes = plt.subplots(len(labels), figsize=(10, 40), sharex=True)
for i in range(14):
    ax = axes[i]
    ax.plot(a[i,:, :], "k", alpha=0.3)
    ax.set_xlim(0, len(a[0,:,0]))
    ax.set_ylabel(labels[i])
    #ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number")
plt.savefig('Graphs/Walkers_LRG_NGCplusSGC')

In [ ]:
from cosmoprimo.fiducial import DESI

cosmo = DESI()

In [ ]:
Likelihood(**chain.choice(index = 'argmax',params=likelihood.varied_params))
observable.plot()
plt.savefig('Graphs/LRG_NGCplusSGC_power_spectrum')
plt.show()